In [66]:
from bs4 import BeautifulSoup
from html2text import html2text
import logging
import re
import requests
import urllib

# from le_utils.constants import content_kinds
from le_utils.constants import licenses
from ricecooker.classes import nodes, files
from ricecooker.classes.files import VideoFile, HTMLZipFile, WebVideoFile, ThumbnailFile
from ricecooker.classes.nodes import ChannelNode, HTML5AppNode, TopicNode, VideoNode, DocumentNode, ExerciseNode
from ricecooker.commands import uploadchannel
from ricecooker.exceptions import UnknownContentKindError, UnknownFileTypeError, raise_for_invalid_channel
from ricecooker.utils.caching import CacheForeverHeuristic, FileCache, CacheControlAdapter
from ricecooker.config import LOGGER

In [2]:
LOGGER.setLevel(logging.INFO)
LOGGER.addHandler(logging.StreamHandler())

sess = requests.Session()
cache = FileCache('.webcache')
basic_adapter = CacheControlAdapter(cache=cache)
forever_adapter= CacheControlAdapter(heuristic=CacheForeverHeuristic(), cache=cache)

sess.mount('http://', basic_adapter)
sess.mount('https://', basic_adapter)
sess.mount('http://www.prathamopenschool.org', forever_adapter)
sess.mount('https://www.prathamopenschool.org', forever_adapter)


In [3]:
BASE_URL = 'http://www.prathamopenschool.org'
LANGUAGE = 'hn'

def get_absolute_path(path):
    return urllib.parse.urljoin(BASE_URL, path)

def get_page(path):
    url = get_absolute_path(path)
    resp = sess.get(url)
    return BeautifulSoup(resp.content, 'html.parser')

def get_source_id(path):
    return path.strip("/").split("/")[-1]


In [72]:
def construct_channel(*args, **kwargs):
    channel = ChannelNode(
        title="Pratham Open School",
        source_domain=BASE_URL,
        source_id="pratham-open-school",
    )
    get_topics(channel, LANGUAGE)
    return channel

In [73]:
def get_topics(parent, path):
    doc = get_page(path)
    try:
        menu_row = doc.find('div', {'id': 'menu-row'})
    except Exception as e:
        print('get_topics: %s : %s' % (e, doc))
    for topic in menu_row.find_all('a'):
        try:
            if topic['href'] == '#':
                continue
            print('topic: %s' % (topic['href']))
            title = topic.get_text().strip()
            source_id = get_source_id(topic['href'])
            node = TopicNode(title=title, source_id=source_id)
            parent.add_child(node)
            get_subtopics(node, topic['href'])
            #break
        except Exception as e:
            print('get_topics: %s : %s' % (e, topic))

In [75]:
def get_subtopics(parent, path):
    doc = get_page(path)
    try:
        menu_row = doc.find('div', {'id' : 'body-row'}).find('div', {'class' : 'col-md-2'})
    except Exception as e:
        print('get_subtopics: %s : %s' % (e, doc))
    subtopics = menu_row.find_all('a')
    for subtopic in subtopics:
        try:
            print('subtopic: %s' % (subtopic['href']))
            title = subtopic.get_text().strip()
            source_id = get_source_id(subtopic['href'])
            node = TopicNode(title=title, source_id=source_id)
            parent.add_child(node)
            get_lessons(node, subtopic['href'])
        except Exception as e:
            print('get_subtopics: %s : %s' % (e, subtopic))

In [76]:
def get_lessons(parent, path):
    doc = get_page(path)
    try:
        menu_row = doc.find('div', {'id' : 'body-row'}).find('div', {'class' : 'col-md-9'})
    except Exception as e:
        print('get_lessons: %s : %s' % (e, doc))
    for lesson in menu_row.find_all('div', {'class' : 'thumbnail'}):
        try:
            title = lesson.find('div', {'class': 'txtline'}).get_text().strip()
            link = lesson.find('a')['href']
            print('lesson: %s' % (link))
            source_id = get_source_id(link)
            node = TopicNode(title=title, source_id=source_id)
            parent.add_child(node)
            get_contents(node, link)
        except Exception as e:
            print('get_lessons: %s : %s' % (e, lesson))

In [77]:
def get_contents(parent, path):
    doc = get_page(path)
    try:
        menu_row = doc.find('div', {'id' : 'row-exu'})
    except Exception as e:
        print('get_contents: %s : %s' % (e, doc))
    for content in menu_row.find_all('div', {'class':'col-md-3'}):
        try:
            title = content.find('div', {'class' : 'txtline'}).get_text()
            link = content.find('a', {'title' : 'Download'})
            if link:
                link = link['href']
            else:
                # let's get it from the onclick attribute
                link = content.find('a', {'id' : 'navigate'})
                regex = re.compile(r"res_click\('(.*)','.*','.*','.*'\)")
                match = regex.search(link['onclick'])
                link = match.group(1)
            if link.endswith('mp4'):
                video = VideoNode(
                    title=title,
                    source_id=get_source_id(link),
                    license = 'Public Domain',
                    files=[VideoFile(get_absolute_path(link))])
                parent.add_child(video)
                #break
        except Exception as e:
            print('get_contents: %s : %s' % (e, content))

topic: /hn/Course/Mathematics
subtopic: /hn/Course/Mathematics/CRS17
lesson: /hn/Lesson/LES55
lesson: /hn/Lesson/LES56
lesson: /hn/Lesson/LES57
lesson: /hn/Lesson/LES58
lesson: /hn/Lesson/LES74
lesson: /hn/Lesson/LES76
subtopic: /hn/Course/Mathematics/CRS98
lesson: /hn/Lesson/LES603
subtopic: /hn/Course/Mathematics/CRS99
lesson: /hn/Lesson/LES604
subtopic: /hn/Course/Mathematics/CRS27
lesson: /hn/Lesson/LES86
lesson: /hn/Lesson/LES87
lesson: /hn/Lesson/LES88
lesson: /hn/Lesson/LES89
lesson: /hn/Lesson/LES90
lesson: /hn/Lesson/LES537
subtopic: /hn/Course/Mathematics/CRS82
lesson: /hn/Lesson/LES509
lesson: /hn/Lesson/LES513
lesson: /hn/Lesson/LES514
lesson: /hn/Lesson/LES515
lesson: /hn/Lesson/LES112
subtopic: /hn/Course/Mathematics/CRS41
lesson: /hn/Lesson/LES107
subtopic: /hn/Course/Mathematics/CRS24
lesson: /hn/Lesson/LES75
subtopic: /hn/Course/Mathematics/CRS21
lesson: /hn/Lesson/LES85
subtopic: /hn/Course/Mathematics/CRS40
lesson: /hn/Lesson/LES100
lesson: /hn/Lesson/LES101
topic: /

      Pratham Open School (ChannelNode): 473 descendants
         गणित (TopicNode): 94 descendants
            ज्यामिती से परिचय (TopicNode): 14 descendants
               त्रिभुज (TopicNode): 2 descendants
                  त्रिभुजों के प्रकार  (VideoNode): 1 file
                  त्रिभुज के मज़ेदार खेल  (VideoNode): 1 file
               कोण (TopicNode): 2 descendants
                  कोण  (VideoNode): 1 file
                  कोण और उनके प्रकार  (VideoNode): 1 file
               वृत्त (TopicNode): 1 descendant
                  वृत्त  (VideoNode): 1 file
               परिमाप (TopicNode): 1 descendant
                  परिमाप  (VideoNode): 1 file
               आकृतियाँ और बहुभुज (TopicNode): 1 descendant
                  आकृतियाँ और बहुभुज  (VideoNode): 1 file
               क्षेत्रफल (TopicNode): 1 descendant
                  क्षेत्रफल  (VideoNode): 1 file
            ब्याज (TopicNode): 4 descendants
               ब्याज (TopicNode): 3 descendants
                  ब्याज की भू

subtopic: /hn/gamelist/CRS104/LES622


                  मनपसंद हीरो  (VideoNode): 1 file
                  नापसंद हीरो  (VideoNode): 1 file
                  पिक्चर देखने जाना  (VideoNode): 1 file
                  पिक्चर पर चर्चा  (VideoNode): 1 file
                  सीमा और मधु की पसंद नापसंद (दोहराएँ)   (VideoNode): 1 file
                  दीपक और राहुल की पसंद नापसंद (दोहराएँ)  (VideoNode): 1 file
               समय को समझे (TopicNode): 6 descendants
                  सुबह हो गयी मामू  (VideoNode): 1 file
                  स्कूल चले हम  (VideoNode): 1 file
                  टाइमपास  (VideoNode): 1 file
                  एक दिन में दो-दो पार्टी  (VideoNode): 1 file
                  चलो रेस लगाये  (VideoNode): 1 file
                  निशा के प्लान - दोहराए  (VideoNode): 1 file
               खरीददारी (TopicNode): 5 descendants
                  खरीददारी  (VideoNode): 1 file
                  रामू काका की दुकान  (VideoNode): 1 file
                  किराने का सामान  (VideoNode): 1 file
                  फैशन पॉइंट - क

                  जोड़ से जुड़े हम  (VideoNode): 1 file
                  रीढ़ को जानें  (VideoNode): 1 file
                  कंकाल की कलाबाज़ियाँ  (VideoNode): 1 file
                  कब्ज़ों वाला आदमी  (VideoNode): 1 file
               रक्त परिसंचरण तंत्र (TopicNode): 3 descendants
                  रक्त परिचय  (VideoNode): 1 file
                  रक्त में होता क्या हैं ?  (VideoNode): 1 file
                  रक्त परिसंचरण तंत्र  (VideoNode): 1 file
            सूर्य और पृथ्वी (TopicNode): 8 descendants
               सौरमंडल की पहचान (TopicNode): 2 descendants
                  सौरमंडल  (VideoNode): 1 file
                  अंतरिक्ष से पृथ्वी  (VideoNode): 1 file
               पृथ्वी का सफ़र (TopicNode): 4 descendants
                  नक्शे पर जगहों को ढूंढना  (VideoNode): 1 file
                  चलो समझे नक्शे को  (VideoNode): 1 file
                  समय समय की वात 1  (VideoNode): 1 file
                  समय समय की वात 2  (VideoNode): 1 file
            ऊर्जा (TopicNode): 6 des

            बेडसाइड असिस्टंट (TopicNode): 9 descendants
               परिचय (TopicNode): 1 descendant
                  बेडसाइड सहायक  (VideoNode): 1 file
               प्रत्यक्ष-कार्य (TopicNode): 4 descendants
                  हाथ धोना  (VideoNode): 1 file
                  बिस्तर बनाने  (VideoNode): 1 file
                  ऑटो क्लेव  (VideoNode): 1 file
                  तापमान  (VideoNode): 1 file
               बेडसाइड असिस्टेंट के कहानियां (TopicNode): 1 descendant
                  एक बिस्तर सहायक की कहानी  (VideoNode): 1 file
         8 वी कक्षा (TopicNode): 61 descendants
            विज्ञान (TopicNode): 27 descendants
               धातु और दुनिया (TopicNode): 1 descendant
                  धातुओं, अधातुओं की प्रतिक्रिया   (VideoNode): 1 file
               ईंधन कैसे बनते है ? (TopicNode): 3 descendants
                  कार्बनीकरण  (VideoNode): 1 file
                  पेट्रोलियम, प्राकृतिक गैस-निर्माण   (VideoNode): 1 file
                  उससे कितना लंबा  (VideoNode):

In [ ]:
construct_channel().print_tree()